In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [102]:
# filename = '/Users/layden/Downloads/HighSHighG_RS_raw_5s_Cad_all_grades.fits'
filename = '/Users/layden/Downloads/HighSHighG_RS_raw_5s_Cad_all_events.fits'
# Get header and data
header = fits.getheader(filename)
data = fits.getdata(filename)
hdu = fits.open(filename)
print(hdu[1].header)
print(data[0])
# Get elements of data for which element[1] is 

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  106 / length of dimension 1                          NAXIS2  =               454014 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    4 / number of table fields                         TTYPE1  = 'FRAME   '                                                            TFORM1  = 'J       '                                                            TZERO1  =           2147483648                                                  TTYPE2  = 'X       '                                                            TFORM2  = 'I       '                    

In [100]:
def get_calibration_vars(mode):
    """Get the calibration variables for the given mode."""
    if mode == "CMS":
        # Coefficients for the CMS calibration. Load from Calibration Data
        coeffs = fits.open('Calibration Data/coeff_array_CMS.fits')[0].data
        transition_value = 3000
        high_cutoff = 60000
        bits = 16
        bias_frame = fits.open('Calibration Data/master_bias_CMS.fits')[0].data
    elif mode == "HSHGRS":
        # Coefficients for the HSHGRS calibration. Load from Calibration Data
        coeffs = fits.open('Calibration Data/coeff_array_HSHGRS.fits')[0].data
        transition_value = 750
        high_cutoff = 15000
        bits = 14
        bias_frame = fits.open('Calibration Data/master_bias_HSHGRS.fits')[0].data
    elif mode == "HSHGRS_spring":
        # Coefficients for the HSHGRS calibration. Load from Calibration Data
        coeffs = fits.open('Calibration Data/coeff_array_HSHGRS.fits')[0].data
        transition_value = 750
        high_cutoff = 15000
        bits = 14
        # bias_frame = fits.open('Calibration Data/master_bias_HSHGRS_spring.fits')[0].data
        # For X-ray data event list, data has already been bias subtracted
        bias_frame = None
    elif mode == "HSHGGS":
        # Coefficients for the HSHGRS calibration. Load from Calibration Data
        coeffs = fits.open('Calibration Data/coeff_array_HSHGGS.fits')[0].data
        transition_value = 750
        high_cutoff = 15000
        bits = 14
        bias_frame = fits.open('Calibration Data/master_bias_HSHGGS.fits')[0].data
    else:
        raise ValueError(f"Invalid mode '{mode}'")
    return coeffs, transition_value, high_cutoff, bits, bias_frame



In [101]:
def correct_roi(roi_image, coeffs, transition_value, high_cutoff, bits, order_low,
                   bias_frame=None, verbose=False):
    """Correct a stack of images using the given coefficients."""
    corrected_stack = np.zeros_like(roi_image, dtype=np.uint16)
    order_high = len(coeffs) - order_low
    if bias_frame is not None:
        roi_image = roi_image - bias_frame
    # Check if roi_image is just one image
    if len(roi_image.shape) == 2:
        roi_image = np.array([roi_image])
        corrected_stack = np.array([corrected_stack])
    for i, frame in enumerate(roi_image):
        t0 = time.time()
        corrected_frame = np.zeros_like(frame, dtype=float)
        frame = frame.astype(float)
        low_vals_frame = np.zeros_like(frame)
        low_vals_frame[frame <= transition_value] = frame[frame <= transition_value]
        low_vals_true_frame = frame <= transition_value
        high_vals_frame = np.zeros_like(frame)
        high_vals_frame[(frame > transition_value) & (frame < high_cutoff)] = frame[(frame > transition_value) & (frame < high_cutoff)]
        high_vals_true_frame = (frame > transition_value) & (frame < high_cutoff)
        saturated_vals_true_frame = frame >= high_cutoff
        for j in range(order_low - 1):
            corrected_frame += coeffs[j] * low_vals_frame ** (order_low - 1 - j)
        for j in range(order_high - 1):
            corrected_frame += coeffs[j + order_low] * high_vals_frame ** (order_high - 1 - j)
            # For saturated values, set the corrected value to the correction of high_cutoff
            corrected_frame += coeffs[j + order_low] * (high_cutoff * saturated_vals_true_frame.astype(float)) ** (order_high - 1 - j)
        # Add the correct constant term
        corrected_frame += coeffs[order_low - 1] * low_vals_true_frame.astype(float)
        corrected_frame += coeffs[-1] * high_vals_true_frame.astype(float)
        corrected_frame += coeffs[-1] * saturated_vals_true_frame.astype(float)
        # Bound corrected_frame between 0 and 2**bits - 1
        corrected_frame[corrected_frame < 0] = 0
        corrected_frame[corrected_frame > 2**bits - 1] = 2**bits - 1
        # Make NaNs equal to 2 ** bits - 1
        corrected_frame[np.isnan(corrected_frame)] = 2 ** bits - 1
        corrected_stack[i] = np.rint(corrected_frame).astype(np.uint16)
        if verbose:
            print('Frame ', i + 1, '/', len(roi_image), ' calibrated with mean value ',
                  format(np.mean(corrected_stack[i]), '.0f'), ' in ',
                  format(time.time() - t0, '3.2f'), ' seconds')
    return corrected_stack

In [103]:
mode = "HSHGRS_spring"
coeff_array, transition_val, high_cutoff, bits, bias_frame = get_calibration_vars(mode)
roi_width = 7
for event in data[:10]:
    start_row, start_col = event[2], event[1]
    event_roi = event[-1]
    coeff_array_roi = coeff_array[:, start_row:start_row + roi_width, start_col:start_col + roi_width]
    corrected_event_roi = correct_roi(event_roi, coeff_array_roi, transition_val, high_cutoff, bits, 12, bias_frame=bias_frame, verbose=True)
    print(bias_frame)
    print(corrected_event_roi)
        

FileNotFoundError: [Errno 2] No such file or directory: 'Calibration Data/coeff_array_HSHGRS.fits'